# Initialization

In [1]:
from pyspark.sql import SparkSession

In [2]:
jdbc_driver_path= "/mnt/c/Asmaa/POSTEGER_DRIVER/postgresql-42.7.5.jar"

In [3]:
spark = SparkSession.builder \
.appName("SalesAnalysis_ETL") \
.config("spark.jars", jdbc_driver_path).getOrCreate()

25/04/10 19:24:19 WARN Utils: Your hostname, Abdelrhman0 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/10 19:24:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/10 19:24:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sparkContext.setLogLevel("OFF")  


In [161]:
spark.sparkContext.setLogLevel("ERROR")


In [ ]:
spark.sparkContext.setLogLevel("WARN")


In [5]:
def read_table(table_name):
    return spark.read.format("jdbc") \
        .option("url", "jdbc:postgresql://localhost:5432/olist") \
        .option("dbtable", table_name) \
        .option("user", "asmaa") \
        .option("password", "123") \
        .option("driver", "org.postgresql.Driver") \
        .load()


# Extract

In [6]:
#Add customer df

df_customers = read_table("wsl.customers")
df_customers.show()

+--------------------+--------------------+------------------------+-------------+--------------+------+---+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|gender|age|marital_status|
+--------------------+--------------------+------------------------+-------------+--------------+------+---+--------------+
|2ad205cfba7e19108...|1458e8048a02a286a...|                    8562|          poa|            SP|  Male| 64|       Married|
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|       franca|            SP|Female| 35|       Married|
|19d2fe3e9e67da50d...|3ff45f223bb4d6fc7...|                   24240|      niteroi|            RJ|Female| 64|       Married|
|229755021cdd010bc...|28a9545ffe21521e8...|                   13306|          itu|            SP|  Male| 37|        Single|
|f04018667ff9194bb...|87e5b2ef0c9dd7a45...|                    8473|    sao paulo|            SP|  Male| 39|        Single|
|1158880

In [7]:
#Add seller df
df_sellers = read_table("wsl.sellers")
df_sellers.show()

+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|c35affa65afd25015...|                  8050|        sao paulo|          SP|
|ceb7b4fb9401cd378...|                 22790|   rio de janeiro|          RJ|
|e9874ae10b2d60d43...|                 86604|         rolandia|          PR|
|da4d149c0ddbac905...|                  9861|              sbc|          SP|
|e0878efa0e0b7e531...|                 80230|         curitiba|          PR|
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|

In [8]:
#Add geo df
df_geo = read_table("wsl.geolocation")
df_geo.show()

+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                       1037|   -23.54562128|   -46.63929205|       sao paulo|               SP|
|                       1046|   -23.54608113|    -46.6448203|       sao paulo|               SP|
|                       1041|   -23.54439216|   -46.63949931|       sao paulo|               SP|
|                       1035|   -23.54157796|   -46.64160722|       sao paulo|               SP|
|                       1012|    -23.5477623|   -46.63536054|       são paulo|               SP|
|                       1047|   -23.54627311|   -46.64122517|       sao paulo|               SP|
|                       1013|   -23.54692321|    -46.6342637|       sao paulo|               SP|
|                       1029| 

# EDA

- show schema

In [104]:
df_geo.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [105]:
df_geo.columns

['geolocation_zip_code_prefix',
 'geolocation_lat',
 'geolocation_lng',
 'geolocation_city',
 'geolocation_state']

In [106]:
df_customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = true)



In [107]:
df_sellers.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



- Show Nulls

In [164]:
from pyspark.sql.functions import col, isnan, when, count

df_customers.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df_customers.columns]).show()


[Stage 369:>                                                        (0 + 1) / 1]

+-----------+------------------+------------------------+-------------+--------------+------+---+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|gender|age|marital_status|
+-----------+------------------+------------------------+-------------+--------------+------+---+--------------+
|          0|                 0|                       0|            0|             0|     0|  0|             0|
+-----------+------------------+------------------------+-------------+--------------+------+---+--------------+



In [109]:
from pyspark.sql.functions import col, isnan, when, count

df_geo.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df_geo.columns]).show()


+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+



In [110]:
df_sellers.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df_sellers.columns]).show()


+---------+----------------------+-----------+------------+
|seller_id|seller_zip_code_prefix|seller_city|seller_state|
+---------+----------------------+-----------+------------+
|        0|                     0|          0|           0|
+---------+----------------------+-----------+------------+



- Summary Statistics

In [111]:
# spark.conf.set("spark.sql.debug.maxToStringFields", 1000)


In [112]:
df_customers.describe().show()


[Stage 208:>                                                        (0 + 1) / 1]

+-------+--------------------+--------------------+------------------------+-------------------+--------------+------+------------------+--------------+
|summary|         customer_id|  customer_unique_id|customer_zip_code_prefix|      customer_city|customer_state|gender|               age|marital_status|
+-------+--------------------+--------------------+------------------------+-------------------+--------------+------+------------------+--------------+
|  count|               99441|               99441|                   99441|              99441|         99441| 99441|             99441|         99441|
|   mean|                null|                null|       35137.47458291851|               null|          null|  null| 43.97144035156525|          null|
| stddev|                null|                null|       29797.93899620611|               null|          null|  null|15.289492896330373|          null|
|    min|00012a2ce6f8dcda2...|0000366f3b9a7992b...|                    1003|abadia

In [113]:
df_sellers.describe().show()


+-------+--------------------+----------------------+---------------+------------+
|summary|           seller_id|seller_zip_code_prefix|    seller_city|seller_state|
+-------+--------------------+----------------------+---------------+------------+
|  count|                3095|                  3095|           3095|        3095|
|   mean|                null|    32291.059450726978|           null|        null|
| stddev|                null|    32713.453829508988|           null|        null|
|    min|0015a82c2db000af6...|                  1001|abadia de goias|          AC|
|    max|ffff564a4f9085cd2...|                 99730|          xaxim|          SP|
+-------+--------------------+----------------------+---------------+------------+



In [114]:
df_geo.describe().show()


+-------+---------------------------+------------------+------------------+----------------+-----------------+
|summary|geolocation_zip_code_prefix|   geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|
+-------+---------------------------+------------------+------------------+----------------+-----------------+
|  count|                      19177|             19177|             19177|           19177|            19177|
|   mean|          42852.03994368254|-18.90105810758131|-45.66892790017814|            null|             null|
| stddev|          30884.10668417943| 7.494320082727903| 6.817455811523201|            null|             null|
|    min|                       1001|      -36.60537441|       -72.9272963| abadia de goias|               AC|
|    max|                      99990|       42.18400274|       121.1053938|          zortea|               TO|
+-------+---------------------------+------------------+------------------+----------------+-----------------+



- Count Distinct Values

In [115]:
df_customers.select("customer_city").distinct().count()


4119

In [116]:
df_customers.select("customer_state").distinct().count()


27

In [117]:
df_customers.select("customer_zip_code_prefix").distinct().count()


14994

In [118]:
df_geo.select("geolocation_zip_code_prefix").distinct().count()


19177

In [119]:
df_geo.select("geolocation_city").distinct().count()


5813

In [120]:
df_sellers.select("seller_zip_code_prefix").distinct().count()


2246

In [121]:
df_sellers.select("seller_city").distinct().count()


609

In [122]:
df_sellers.select("seller_state").distinct().count()


23

- Aggregation

In [123]:
df_customers.groupBy("Gender").agg({"Age": "mean"}).show()


+------+------------------+
|Gender|          avg(Age)|
+------+------------------+
|Female|43.948033670214194|
|  Male|44.041755586873364|
+------+------------------+



In [124]:
df_customers.groupBy("customer_unique_id").agg({"customer_id": "count"}).show()


+--------------------+------------------+
|  customer_unique_id|count(customer_id)|
+--------------------+------------------+
|861eff4711a542e4b...|                 1|
|e607ede0e63436308...|                 1|
|28da048f094c0c9cb...|                 1|
|212c759d8c4f2d4d9...|                 1|
|f96176e892232662d...|                 1|
|000c8bdb58a29e711...|                 1|
|9ccbb5f759db041b2...|                 1|
|4b384b778ebc0449d...|                 2|
|4df43d4c7d3a093a5...|                 1|
|4be1583defacacfea...|                 1|
|c85df1c6ef6f7bb60...|                 2|
|1a3d1e8b28c0ee27c...|                 1|
|2a6ef69674d6f2800...|                 1|
|2d6d1699603a346a2...|                 1|
|5b71ca52f13f79c2b...|                 2|
|9a176d6883a469743...|                 1|
|dfe634d4b8e067879...|                 1|
|5f439c10be261b762...|                 1|
|cbe71e4cb0b82b97b...|                 2|
|782d02110160b9efd...|                 1|
+--------------------+------------

In [125]:
df_customers.groupBy("customer_city").count().show()


+--------------------+-----+
|       customer_city|count|
+--------------------+-----+
|            camacari|   61|
|           arapiraca|   29|
|           itaberaba|   10|
|           igrejinha|   17|
|  aguas de sao pedro|    3|
|            vermelho|    1|
|                pote|    3|
|jijoca de jericoa...|    4|
|            barracao|    3|
|                iepe|    5|
|divino das laranj...|    1|
|divino de sao lou...|    1|
|              bacaxa|    1|
|   redencao da serra|    1|
|            itanhaem|   80|
|       astolfo dutra|   11|
|             brusque|   60|
|           boa vista|   45|
|  cachoeira paulista|   25|
|            guaranta|    5|
+--------------------+-----+
only showing top 20 rows



In [126]:
df_customers.groupBy("customer_city").count().orderBy("count", ascending=False).show()


+--------------------+-----+
|       customer_city|count|
+--------------------+-----+
|           sao paulo|15540|
|      rio de janeiro| 6882|
|      belo horizonte| 2773|
|            brasilia| 2131|
|            curitiba| 1521|
|            campinas| 1444|
|        porto alegre| 1379|
|            salvador| 1245|
|           guarulhos| 1189|
|sao bernardo do c...|  938|
|             niteroi|  849|
|         santo andre|  797|
|              osasco|  746|
|              santos|  713|
|             goiania|  692|
| sao jose dos campos|  691|
|           fortaleza|  654|
|            sorocaba|  633|
|              recife|  613|
|       florianopolis|  570|
+--------------------+-----+
only showing top 20 rows



In [127]:
df_customers.filter(df_customers.customer_city.isin("sao paulo", "São Paulo")) \
    .groupBy("customer_city").count() \
    .orderBy("count", ascending=False).show()


+-------------+-----+
|customer_city|count|
+-------------+-----+
|    sao paulo|15540|
+-------------+-----+



In [128]:
df_geo.filter(df_geo.geolocation_city.isin("sao paulo", "são paulo")) \
    .groupBy("geolocation_city").count() \
    .orderBy("count", ascending=False).show()

#did not see it too -->> because case sensitive now ok!
#'ã' --> need to transform all this letters

+----------------+-----+
|geolocation_city|count|
+----------------+-----+
|       sao paulo| 3183|
+----------------+-----+



In [129]:
df_customers.filter(df_customers.customer_city.isin("sao paulo", "são paulo")) \
    .groupBy("customer_city").count() \
    .orderBy("count", ascending=False).show()


+-------------+-----+
|customer_city|count|
+-------------+-----+
|    sao paulo|15540|
+-------------+-----+



In [130]:
df_sellers.filter(df_sellers.seller_city.isin("sao paulo", "são paulo")) \
    .groupBy("seller_city").count() \
    .orderBy("count", ascending=False).show()


+-----------+-----+
|seller_city|count|
+-----------+-----+
|  sao paulo|  695|
+-----------+-----+



In [131]:
df_customers.groupBy("Gender").count().show()


+------+-----+
|Gender|count|
+------+-----+
|Female|74606|
|  Male|24835|
+------+-----+



In [132]:
df_customers.groupBy("customer_unique_id").agg({"customer_id": "count"}).show()


+--------------------+------------------+
|  customer_unique_id|count(customer_id)|
+--------------------+------------------+
|861eff4711a542e4b...|                 1|
|e607ede0e63436308...|                 1|
|28da048f094c0c9cb...|                 1|
|212c759d8c4f2d4d9...|                 1|
|f96176e892232662d...|                 1|
|000c8bdb58a29e711...|                 1|
|9ccbb5f759db041b2...|                 1|
|4b384b778ebc0449d...|                 2|
|4df43d4c7d3a093a5...|                 1|
|4be1583defacacfea...|                 1|
|c85df1c6ef6f7bb60...|                 2|
|1a3d1e8b28c0ee27c...|                 1|
|2a6ef69674d6f2800...|                 1|
|2d6d1699603a346a2...|                 1|
|5b71ca52f13f79c2b...|                 2|
|9a176d6883a469743...|                 1|
|dfe634d4b8e067879...|                 1|
|5f439c10be261b762...|                 1|
|cbe71e4cb0b82b97b...|                 2|
|782d02110160b9efd...|                 1|
+--------------------+------------

In [133]:
customer_by_zip_df = df_customers.groupBy("customer_zip_code_prefix").agg(count("customer_id").alias("customer_count"))
customer_by_zip_df.show()

+------------------------+--------------+
|customer_zip_code_prefix|customer_count|
+------------------------+--------------+
|                   29285|            17|
|                   20735|            37|
|                   18051|            22|
|                   13840|            12|
|                   12940|            21|
|                    6397|             4|
|                   26087|            15|
|                   70355|             3|
|                   74820|             5|
|                   31035|            30|
|                    7240|            16|
|                    4935|             3|
|                   18800|            19|
|                   95080|             8|
|                    2122|             7|
|                    4101|             5|
|                   62680|             5|
|                    2366|             4|
|                   89874|            10|
|                   66010|             2|
+------------------------+--------

# Transformation

- Customer transformation

In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

#Add SK column
windowSpec = Window.orderBy("customer_id")
df_with_sk = df_customers.withColumn("customer_SK", row_number().over(windowSpec))
df_with_sk.show()


+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-----------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|gender|age|marital_status|customer_SK|
+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-----------+
|00012a2ce6f8dcda2...|248ffe10d632bebe4...|                    6273|              osasco|            SP|Female| 46|       Married|          1|
|000161a058600d590...|b0015e09bb4b6e47c...|                   35550|         itapecerica|            MG|  Male| 57|       Married|          2|
|0001fd6190edaaf88...|94b11d37cd61cb299...|                   29830|        nova venecia|            ES|Female| 40|       Married|          3|
|0002414f953443074...|4893ad4ea28b2c5b3...|                   39664|            mendonca|            MG|Female| 26|       Married|          4|

In [11]:
# Get all column names, move 'sk' to the front
cols = df_with_sk.columns
cols = ['customer_SK'] + [col for col in cols if col != 'customer_SK']

# Reorder the DataFrame
df_reordered = df_with_sk.select(cols)
df_reordered.show()


+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+
|customer_SK|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|gender|age|marital_status|
+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+
|          1|00012a2ce6f8dcda2...|248ffe10d632bebe4...|                    6273|              osasco|            SP|Female| 46|       Married|
|          2|000161a058600d590...|b0015e09bb4b6e47c...|                   35550|         itapecerica|            MG|  Male| 57|       Married|
|          3|0001fd6190edaaf88...|94b11d37cd61cb299...|                   29830|        nova venecia|            ES|Female| 40|       Married|
|          4|0002414f953443074...|4893ad4ea28b2c5b3...|                   39664|            mendonca|            MG|Female| 26|       Married|

In [12]:
from pyspark.sql.functions import when, col, rand
from pyspark.sql.types import IntegerType
import random

In [13]:
#Map abbreviations to full state names
df_full_state = df_reordered.withColumn("customer_state_full",
    when(col("customer_state") == "AC", "Acre")
    .when(col("customer_state") == "AL", "Alagoas")
    .when(col("customer_state") == "AP", "Amapa")
    .when(col("customer_state") == "AM", "Amazonas")
    .when(col("customer_state") == "BA", "Bahia")
    .when(col("customer_state") == "CE", "Ceara")
    .when(col("customer_state") == "DF", "Distrito Federal")
    .when(col("customer_state") == "ES", "Espirito Santo")
    .when(col("customer_state") == "GO", "Goias")
    .when(col("customer_state") == "MA", "Maranhao")
    .when(col("customer_state") == "MT", "Mato Grosso")
    .when(col("customer_state") == "MS", "Mato Grosso do Sul")
    .when(col("customer_state") == "MG", "Minas Gerais")
    .when(col("customer_state") == "PA", "Para")
    .when(col("customer_state") == "PB", "Paraiba")
    .when(col("customer_state") == "PR", "Parana")
    .when(col("customer_state") == "PE", "Pernambuco")
    .when(col("customer_state") == "PI", "Piaui")
    .when(col("customer_state") == "RJ", "Rio de Janeiro")
    .when(col("customer_state") == "RN", "Rio Grande do Norte")
    .when(col("customer_state") == "RS", "Rio Grande do Sul")
    .when(col("customer_state") == "RO", "Rondonia")
    .when(col("customer_state") == "RR", "Roraima")
    .when(col("customer_state") == "SC", "Santa Catarina")
    .when(col("customer_state") == "SP", "Sao Paulo")
    .when(col("customer_state") == "SE", "Sergipe")
    .when(col("customer_state") == "TO", "Tocantins")
)

In [14]:
df_full_state.show()

+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+
|customer_SK|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|gender|age|marital_status|customer_state_full|
+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+
|          1|00012a2ce6f8dcda2...|248ffe10d632bebe4...|                    6273|              osasco|            SP|Female| 46|       Married|          Sao Paulo|
|          2|000161a058600d590...|b0015e09bb4b6e47c...|                   35550|         itapecerica|            MG|  Male| 57|       Married|       Minas Gerais|
|          3|0001fd6190edaaf88...|94b11d37cd61cb299...|                   29830|        nova venecia|            ES|Female| 40|       Married|     Espirito Santo|
|          4|0002414f9

In [15]:
!pip install unidecode


In [16]:
from unidecode import unidecode
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


In [17]:
def remove_accents(text):
    if text is not None:
        return unidecode(text)
    return None

remove_accents_udf = udf(remove_accents, StringType())


In [18]:
#To validate the other cities  :

df_full_state = df_full_state.withColumn("customer_city", remove_accents_udf("customer_city"))

In [19]:
df_full_state.filter(df_full_state.customer_city.isin("sao paulo", "são paulo")) \ 
    .groupBy("customer_city").count() \
    .orderBy("count", ascending=False).show()


SyntaxError: unexpected character after line continuation character (1947774362.py, line 1)

- Geo df transformation

In [20]:
df_geo = df_geo.withColumn("geolocation_city", remove_accents_udf("geolocation_city"))


In [21]:
df_geo.filter(df_geo.geolocation_city.isin("sao paulo", "são paulo")) \
    .groupBy("geolocation_city").count() \
    .orderBy("count", ascending=False).show()
#this worked

+----------------+-----+
|geolocation_city|count|
+----------------+-----+
|       sao paulo| 3183|
+----------------+-----+



In [22]:
# Join Customer(after transformation) with geo
joined_customer_geo_df = df_full_state.join(df_geo.select('geolocation_zip_code_prefix','geolocation_lat','geolocation_lng'), df_full_state["customer_zip_code_prefix"] == df_geo["geolocation_zip_code_prefix"], "left")


In [23]:
joined_customer_geo_df.show()

+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------------------+---------------+---------------+
|customer_SK|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|gender|age|marital_status|customer_state_full|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|
+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------------------+---------------+---------------+
|          8|00046a560d407e99b...|0b5295fc9819d831f...|                   20540|      rio de janeiro|            RJ|Female| 55|       Married|     Rio de Janeiro|                      20540|   -22.92173311|   -43.25868934|
|         14|00072d033fe2e5906...|b7c13491fd2aecd93...|                   45026|vitoria da conquista|       

In [24]:
joined_customer_geo_df = joined_customer_geo_df.drop("geolocation_zip_code_prefix")


In [25]:
joined_customer_geo_df.show()

+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------+---------------+
|customer_SK|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|gender|age|marital_status|customer_state_full|geolocation_lat|geolocation_lng|
+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------+---------------+
|          8|00046a560d407e99b...|0b5295fc9819d831f...|                   20540|      rio de janeiro|            RJ|Female| 55|       Married|     Rio de Janeiro|   -22.92173311|   -43.25868934|
|         14|00072d033fe2e5906...|b7c13491fd2aecd93...|                   45026|vitoria da conquista|            BA|Female| 62|        Single|              Bahia|    -14.8893525|   -40.84248825|
|          7|000419c54941

In [26]:
sorted_df = joined_customer_geo_df.orderBy("customer_SK")
sorted_df.show()

+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------+---------------+
|customer_SK|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|gender|age|marital_status|customer_state_full|geolocation_lat|geolocation_lng|
+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------+---------------+
|          1|00012a2ce6f8dcda2...|248ffe10d632bebe4...|                    6273|              osasco|            SP|Female| 46|       Married|          Sao Paulo|   -23.49968872|   -46.76690136|
|          2|000161a058600d590...|b0015e09bb4b6e47c...|                   35550|         itapecerica|            MG|  Male| 57|       Married|       Minas Gerais|   -20.47058243|   -45.10871624|
|          3|0001fd6190ed

In [27]:
#Cheak Nulls
sorted_df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in sorted_df.columns]).show()

NameError: name 'count' is not defined

In [28]:
#Add SCD
from pyspark.sql.functions import lit, current_date

df_customers_scd = sorted_df.withColumn("start_date", current_date()) \
                   .withColumn("end_date", lit(None).cast("date")) \
                   .withColumn("is_current", lit(True))

df_customers_scd.show()


+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------+---------------+----------+--------+----------+
|customer_SK|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|gender|age|marital_status|customer_state_full|geolocation_lat|geolocation_lng|start_date|end_date|is_current|
+-----------+--------------------+--------------------+------------------------+--------------------+--------------+------+---+--------------+-------------------+---------------+---------------+----------+--------+----------+
|          1|00012a2ce6f8dcda2...|248ffe10d632bebe4...|                    6273|              osasco|            SP|Female| 46|       Married|          Sao Paulo|   -23.49968872|   -46.76690136|2025-04-10|    null|      true|
|          2|000161a058600d590...|b0015e09bb4b6e47c...|                   35550|         itapece

- sellers Transformation

In [29]:
#Map abbreviations to full state names using `when`
df_sellers_full_state = df_sellers.withColumn("seller_state_full",
    when(col("seller_state") == "AC", "Acre")
    .when(col("seller_state") == "AL", "Alagoas")
    .when(col("seller_state") == "AP", "Amapa")
    .when(col("seller_state") == "AM", "Amazonas")
    .when(col("seller_state") == "BA", "Bahia")
    .when(col("seller_state") == "CE", "Ceara")
    .when(col("seller_state") == "DF", "Distrito Federal")
    .when(col("seller_state") == "ES", "Espirito Santo")
    .when(col("seller_state") == "GO", "Goias")
    .when(col("seller_state") == "MA", "Maranhao")
    .when(col("seller_state") == "MT", "Mato Grosso")
    .when(col("seller_state") == "MS", "Mato Grosso do Sul")
    .when(col("seller_state") == "MG", "Minas Gerais")
    .when(col("seller_state") == "PA", "Para")
    .when(col("seller_state") == "PB", "Paraiba")
    .when(col("seller_state") == "PR", "Parana")
    .when(col("seller_state") == "PE", "Pernambuco")
    .when(col("seller_state") == "PI", "Piaui")
    .when(col("seller_state") == "RJ", "Rio de Janeiro")
    .when(col("seller_state") == "RN", "Rio Grande do Norte")
    .when(col("seller_state") == "RS", "Rio Grande do Sul")
    .when(col("seller_state") == "RO", "Rondonia")
    .when(col("seller_state") == "RR", "Roraima")
    .when(col("seller_state") == "SC", "Santa Catarina")
    .when(col("seller_state") == "SP", "Sao Paulo")
    .when(col("seller_state") == "SE", "Sergipe")
    .when(col("seller_state") == "TO", "Tocantins")
)

In [30]:
df_sellers_full_state.show()

+--------------------+----------------------+-----------------+------------+-----------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|seller_state_full|
+--------------------+----------------------+-----------------+------------+-----------------+
|c35affa65afd25015...|                  8050|        sao paulo|          SP|        Sao Paulo|
|ceb7b4fb9401cd378...|                 22790|   rio de janeiro|          RJ|   Rio de Janeiro|
|e9874ae10b2d60d43...|                 86604|         rolandia|          PR|           Parana|
|da4d149c0ddbac905...|                  9861|              sbc|          SP|        Sao Paulo|
|e0878efa0e0b7e531...|                 80230|         curitiba|          PR|           Parana|
|3442f8959a84dea7e...|                 13023|         campinas|          SP|        Sao Paulo|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|        Sao Paulo|
|ce3ad9de960102d06...|                 20031|   ri

In [31]:
# Join Sellers(after transformation) with geo
joined_sellers_geo_df = df_sellers_full_state.join(df_geo.select('geolocation_zip_code_prefix','geolocation_lat','geolocation_lng'), df_sellers_full_state["seller_zip_code_prefix"] == df_geo["geolocation_zip_code_prefix"], "left")


In [32]:
joined_sellers_geo_df.show()

+--------------------+----------------------+-----------------+------------+-----------------+---------------------------+---------------+---------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|seller_state_full|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|
+--------------------+----------------------+-----------------+------------+-----------------+---------------------------+---------------+---------------+
|8bd0f31cf0a614c65...|                  1222|        sao paulo|          SP|        Sao Paulo|                       1222|   -23.54715958|   -46.64989461|
|768a86e36ad6aae3d...|                  1529|        sao paulo|          SP|        Sao Paulo|                       1529|   -23.56935316|   -46.63690014|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|        Sao Paulo|                       4195|   -23.65725018|   -46.61075945|
|05a48cc8859962767...|                  5372|        sao paulo|       

In [33]:
joined_sellers_geo_df = joined_sellers_geo_df.drop("geolocation_zip_code_prefix")
joined_sellers_geo_df.show()


+--------------------+----------------------+-----------------+------------+-----------------+---------------+---------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|seller_state_full|geolocation_lat|geolocation_lng|
+--------------------+----------------------+-----------------+------------+-----------------+---------------+---------------+
|8bd0f31cf0a614c65...|                  1222|        sao paulo|          SP|        Sao Paulo|   -23.54715958|   -46.64989461|
|768a86e36ad6aae3d...|                  1529|        sao paulo|          SP|        Sao Paulo|   -23.56935316|   -46.63690014|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|        Sao Paulo|   -23.65725018|   -46.61075945|
|05a48cc8859962767...|                  5372|        sao paulo|          SP|        Sao Paulo|   -23.58316603|   -46.76124388|
|c35affa65afd25015...|                  8050|        sao paulo|          SP|        Sao Paulo|   -23.51101953| 

In [34]:
#Add SK column to sellers df
windowSpec_sellers = Window.orderBy("seller_id")
df_sellers_with_sk = joined_sellers_geo_df.withColumn("Seller_SK", row_number().over(windowSpec_sellers))
df_sellers_with_sk.show()

+--------------------+----------------------+--------------------+------------+-----------------+---------------+---------------+---------+
|           seller_id|seller_zip_code_prefix|         seller_city|seller_state|seller_state_full|geolocation_lat|geolocation_lng|Seller_SK|
+--------------------+----------------------+--------------------+------------+-----------------+---------------+---------------+---------+
|0015a82c2db000af6...|                  9080|         santo andre|          SP|        Sao Paulo|   -23.64443891|   -46.53988531|        1|
|001cca7ae9ae17fb1...|                 29156|           cariacica|          ES|   Espirito Santo|   -20.29753658|   -40.40086854|        2|
|001e6ad469a905060...|                 24754|         sao goncalo|          RJ|   Rio de Janeiro|   -22.87403983|    -43.0330893|        3|
|002100f778ceb8431...|                 14405|              franca|          SP|        Sao Paulo|   -20.50679363|   -47.40773764|        4|
|003554e2dce176b55..

In [35]:
# Get all column names, move 'sk' to the front
cols = df_sellers_with_sk.columns
cols = ['Seller_SK'] + [col for col in cols if col != 'Seller_SK']

# Reorder the DataFrame
df_reordered_sellers = df_sellers_with_sk.select(cols)
df_reordered_sellers.show()

+---------+--------------------+----------------------+--------------------+------------+-----------------+---------------+---------------+
|Seller_SK|           seller_id|seller_zip_code_prefix|         seller_city|seller_state|seller_state_full|geolocation_lat|geolocation_lng|
+---------+--------------------+----------------------+--------------------+------------+-----------------+---------------+---------------+
|        1|0015a82c2db000af6...|                  9080|         santo andre|          SP|        Sao Paulo|   -23.64443891|   -46.53988531|
|        2|001cca7ae9ae17fb1...|                 29156|           cariacica|          ES|   Espirito Santo|   -20.29753658|   -40.40086854|
|        3|001e6ad469a905060...|                 24754|         sao goncalo|          RJ|   Rio de Janeiro|   -22.87403983|    -43.0330893|
|        4|002100f778ceb8431...|                 14405|              franca|          SP|        Sao Paulo|   -20.50679363|   -47.40773764|
|        5|003554e2d

In [36]:
#Add SCD
df_sellers_scd = df_reordered_sellers.withColumn("start_date", current_date()) \
                   .withColumn("end_date", lit(None).cast("date")) \
                   .withColumn("is_current", lit(True))

df_sellers_scd.show()



+---------+--------------------+----------------------+--------------------+------------+-----------------+---------------+---------------+----------+--------+----------+
|Seller_SK|           seller_id|seller_zip_code_prefix|         seller_city|seller_state|seller_state_full|geolocation_lat|geolocation_lng|start_date|end_date|is_current|
+---------+--------------------+----------------------+--------------------+------------+-----------------+---------------+---------------+----------+--------+----------+
|        1|0015a82c2db000af6...|                  9080|         santo andre|          SP|        Sao Paulo|   -23.64443891|   -46.53988531|2025-04-10|    null|      true|
|        2|001cca7ae9ae17fb1...|                 29156|           cariacica|          ES|   Espirito Santo|   -20.29753658|   -40.40086854|2025-04-10|    null|      true|
|        3|001e6ad469a905060...|                 24754|         sao goncalo|          RJ|   Rio de Janeiro|   -22.87403983|    -43.0330893|2025-0

# load

- show final schema

In [39]:
#customer
df_customers_scd.printSchema()

root
 |-- customer_SK: integer (nullable = false)
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- customer_state_full: string (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- start_date: date (nullable = false)
 |-- end_date: date (nullable = true)
 |-- is_current: boolean (nullable = false)



In [40]:
#seller
df_sellers_scd.printSchema()

root
 |-- Seller_SK: integer (nullable = false)
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)
 |-- seller_state_full: string (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- start_date: date (nullable = false)
 |-- end_date: date (nullable = true)
 |-- is_current: boolean (nullable = false)



- df to load:

In [ ]:
df_customers_scd

In [ ]:
df_sellers_scd